# 1.2.1.Add_Background

In [1]:
import os
import csv
import numpy as np
from PIL import Image
# making tripo maps
import cv2

In [2]:
label_ids = ['10_espalda', '10_frente',
             '20_espalda', '20_frente',
             '50_espalda', '50_frente',
             '100_espalda', '100_frente',
             '200_espalda', '200_frente'
            ]

width = 224
height = 224

#sizes = [0.116, 0.07, 0.05]
sizes = [0.116]
count_per_size = 1

In [3]:
count_background = 0

for label_id in label_ids:
    label = label_id
    prefix_filename = label + "_"

    path_images_labels = 'data_augmentation/banknotes/' + label + '/'
    path_images_backgrounds = 'data_augmentation/background_' + str(width) + '/'
    path_images_labeled = 'data_' + str(width) + '/train/' + label + '/'

    bkg_images = [f for f in os.listdir(path_images_backgrounds) if not f.startswith(".")]
    obj_images = [f for f in os.listdir(path_images_labels) if not f.startswith(".")]

    coordinates  = [] # store coordinate here

    print("Labeling images for label: ", label)
    print("...")

    # Helper functions
    def get_obj_positions(obj, bkg, count=1):
        obj_w, obj_h = [], []
        x_positions, y_positions = [], []
        bkg_w, bkg_h = bkg.size
        # Rescale our obj to have a couple different sizes
        obj_sizes = [tuple([int(s*x) for x in obj.size]) for s in sizes]
        for w, h in obj_sizes:
            obj_w.extend([w]*count)
            obj_h.extend([h]*count)
            max_x, max_y = bkg_w-w, bkg_h-h
            if max_x < w:
                x_positions.extend([0])
            else:
                x_positions.extend(list(np.random.randint(0, max_x, count)))
            if max_y < h:
                y_positions.extend([0])
            else:
                y_positions.extend(list(np.random.randint(0, max_y, count)))
        return obj_h, obj_w, x_positions, y_positions

    # Creando los bounding boxes
    n = 0
    index_selected = 0
    
    #for bkg in bkg_images:
    for img_path in obj_images:
        
        # validate is count_background is mayor than len background
        if count_background >= len(bkg_images):
            count_background = 0
        
        # Load the background image
        bkg_path = path_images_backgrounds + bkg_images[count_background]
        bkg_img = Image.open(bkg_path)
        bkg_x, bkg_y = bkg_img.size
        
        count_background += 1

        # Do single objs first
        #if index_selected >= len(obj_images):
        #    index_selected = 0
        #i = obj_images[index_selected] # seleccionando una imagen
        
        i = img_path # seleccionando una imagen
        index_selected += 1 # Aumentando en uno para la siguiente imagen
        # Load the single obj
        i_path = path_images_labels + i
        obj_img = Image.open(i_path)

        # Get an array of random obj positions (from top-left corner)
        obj_h, obj_w, x_center_pos, y_center_pos = get_obj_positions(obj=obj_img, bkg=bkg_img, count=count_per_size)
        
        # Create synthetic images based on positions
        for h, w, x_center, y_center in zip(obj_h, obj_w, x_center_pos, y_center_pos):
            # Copy background
            bkg_w_obj = bkg_img.copy()
            # Adjust obj size
            new_obj = obj_img.resize(size=(w, h))
            # Paste on the obj
            bkg_w_obj.paste(new_obj, (x_center, y_center), new_obj)
            filename = prefix_filename + str(n)
            output_fp = path_images_labeled + label + filename + '.png'
            # Save the image
            bkg_w_obj.save(fp=output_fp, format="png")
            n += 1

print("finished")

Labeling images for label:  10_espalda
...
Labeling images for label:  10_frente
...
Labeling images for label:  20_espalda
...
Labeling images for label:  20_frente
...
Labeling images for label:  50_espalda
...
Labeling images for label:  50_frente
...
Labeling images for label:  100_espalda
...
Labeling images for label:  100_frente
...
Labeling images for label:  200_espalda
...
Labeling images for label:  200_frente
...
finished
